# Using a basic topic model to correct for background words in peers

One of the richest sources of information regarding a business and its potential peers is the company description. Unfortunately, using a basic tf-idf reweighting of terms does not immediately result in good results, as the absence of cetain words discount a company unnecessarily when measuring its viability.

An alternative to the tf-idf heuristic is to adopt a model-based approach, whereby we assume that each document is generated by some probabilistic model. This has the advantage of allowing us more control over the re-weighting of our terms (through smoothing) that should hopefully result in better document vectors that can better capture relevant meaning from company descriptions. Our hope is that such a model can provide an elegant way to smooth document-specific distributions in order to aid probabilistic peer suggestions.

### LDA-like bayesian model

A statistically elegant way to promote relevant words given that we know which industry each company belongs to can be as follows.

![plate_notation](simple_lda.png)

It assumes that words are generated from ungiram language models, either a generic background model $\phi_{BG}$ that each document is allowed to sample from, or an industry-specific model $\phi_{I}$ that only documents pertaining to that industry can sample from. The probability of sampling from either the background model or the industry model is given by a document-specific parameter $\theta_d$. It is a single number indicating, for a document $d$, the probability that we sample a word from the background model or the industry specific model.

Therefore, to generate a document $d$, we first sample $\theta_d$ from some Beta prior parametrized by $\mathbf{\alpha}$. Then, for each word $w$, we flip a biased coin whose chance of success is $\theta_d$, obtaining an
indicator $z_w$. Then if $z_w$ is zero, we sample a word from $\phi_{BG}$, else we sample a word from $\phi_{m[d]}$, where $m[d]$ indicates which industry $d$ belongs to. We proceede in this fashion until we've sampled all words.

### Inference

The above model is fairly simple and we can immediately do inference on it using e.g. MCMC. However, given the volume
of data we have and the fact that MCMC loses all guarantees of sationarity if trained in a mini-batch fashion, we elected to use variational inference for it. As will be shown below, careful treatment of the E and M steps can allow us to train this model in an embarrassingly parallel fashion without losing any mathematical guarantees of convergence. We also show an implementation of variational inference and show that it converges to apparently sensible topics on real company data. 

We want to maximise the objective.

$$ L = P(\mathbf{X}|\phi, \mathbf{\alpha}) $$

, where $X$ is our documents in bag-of-words format and $\mathbf{\alpha}$ is the prior over $\theta_d$.

We can do this by using the EM algorithm, where at the E-step we aim to find $Q$ such that

$$ \mathbf{\mathbb{E}}_{Q(z,\theta)} P(\mathbf{X},\mathbf{Z}, \mathbf{\theta}) = $$
$$ \mathbf{\mathbb{E}}_{Q(z,\theta)} \left[ \sum_{d}\left[ \sum_{t=0}^1(\alpha_t-1)log\theta_{dt} + \sum_{i=0}^{V} count_d(i) \sum_{t=0}^1 [z_{di}=t](log\phi_{m[d]}(i) + log\theta_{dt}\right]\right] $$

is maximised. Where $[z_{di}=t]$ indicates whether word $i$ of document $d$ comes from $t$ (which can be either the background model or the industry-specific model), $V$ is the size of the vocab, and $count_d(i)$ is the count of word $i$ in document $d$.

We can assume a factorised form for $Q(\mathbf{z},\mathbf{\theta}) = q(\mathbf{z})q(\mathbf{\theta})$ and then use the meanfield approximation to compute $q(\mathbf{z})$ and $q(\mathbf{\theta})$ iteratively.

Thus, we get

$$q(\theta_d) = \mathcal{B}(\alpha_0 + \sum_{i=0}^V count_d(i)\gamma_{di}(0), \alpha_1 + \sum_{i=0}^V count_d(i)\gamma_{di}(1))$$

, is a beta distribution . We use $\gamma_{di}(t)$ to denote $\mathbf{\mathbb{E}}[z_{di} = t]$, and can be interpreted as our belief that word $i$ of document $d$ came from topic $t=0,1$, 
either the background or the industry-specific topic respectively.

Similarly,

$$q(z_{di} = 0) \propto \phi_{BG}(i)exp(\psi(a_d)-\psi(a_d+b_d)) = \gamma_{di}(0)$$
$$q(z_{di} = 1) \propto \phi_{m[d]}(i)exp(\psi(a_d+b_d)-\psi(a_d)) = \gamma_{di}(1)$$

Where $a_d, b_d$ represent the shape parameters of the beta distribution for $q(\theta_d)$.

Thus, the E-step can be implemented as follows:

    Randomly initialise $q(\mathbf{\theta})$ 
    While the change in $q(\mathbf{\theta})$ is >= 1e-5 do:
   
        Update $q(z_{di})$ for each document and word
        Update $q(\mathbf{\theta})$

Unsurprisingly, these equations are very similar to vanilla LDA updates.

In the M-step, we use the $q(\mathbf{z})$ and $q(\mathbf{\theta})$ computed in the E-step to maximise 
$$ \mathbf{\mathbb{E}}_{q(z)q(\theta)} P(\mathbf{X},\mathbf{Z}, \mathbf{\theta})$$ w.r.t. $\phi$, under the restriction that $\sum_{i=0}^V\phi_t(i) = 1$ for all $t$ and $\phi_t(i) \geq 0$ for all $i$ and $t$.

Thus, we obtain the following update:

$$ \phi_t(i) = \frac{\sum_{d, m[d]=t} count_d(i)\gamma_d(t)}{\sum_{i}\sum_{d, m[d]=t} count_d(i)\gamma_d(t)}$$

, which is in fact identical to vanilla LDA with the exception that sums are over all documents which have a certain industry (e.g. only companies whose industry is automotive will be considered in the above sum when we look at the automotive industry topic). A thing to note is that all documents are considered when we're estimating the background topic, so it will naturally converge to containing words that exist in all documents. This becomes apparent in the below results.

### Implementation, Results and discussion

The variational version of the algorithm is implemented [here](simple_lda.py). One implementation detail to note, that allows us to parallelise the E-step is the following. We notice that for updating $\phi$, we only need the sum over all documents of $\gamma$. Now, there is no reason for us to not accumulate this sum from batches of documents into a set of sufficient statistics, which we can then use the update $\phi$ post-hoc.

To better explain this, let's assume we have a huge corpus of documents, $D$. We can split this into arbitrarily small chunks $D_s$, that can be distributed to worker cores. They will execute the E-step for each $D_s$ and return
the sum of $\gamma_d(i)$ over all $d\in D_s$, into a $t\times V$ vector $\Gamma_s$. The trick is that we are free to
sum these $\Gamma_s$ for all batches $s$ as they become available, and compute $\phi$ once all $\Gamma_s$ are done.
This is mathematically equivalent to having performed the E-step over all of $D$ on a single machine. Therefore, the
E-step is embarrassingly parallel, and indeed we provide an option to perform inference using this neat property
using spark.

#### Results

Using the above trick, we managed to train a model over 3 million companies with a business description. We did this [here](company_simpleLDA.ipynb). Also in that notebook you can find the most likely 20 words for each industry-specific and the background topic.

#### Usage in peers ranking

At the moment, suggested peers relies on ElasticSearch's `more_like_this` query, which unfortunately does not always
work well in practice, sometimes even failing to retrieve any peers at all for certain companies. This seems to be because of the absence of certain keywords with a high tf-idf score from certain peers. The aim of this model is to apply smoothing to help overcome this limitation.

More precisely, we aim to enhance the word distribution (obtained from counts) of a certain company as follows:

$$p(w|d) = \frac{count_d(w) + \mu p(w|t)}{|d| + \mu}$$, where $p(w|t)$ is given by $\phi_t(w)$ with $t$ being the
topic corresponding to the industry of $t$.

We can then rank documents based on the KL divergence of $p(w|d)$ between them. This should help correct for cases
where the absence of spurious words causes certain suitable peers to be discarded by `more_like_this`.